In [ ]:
import os
import mlflow
from dotenv import load_dotenv
from src.old_graph import create_graph

In [5]:
%pwd

'/Users/henil08/College/Semesters/Fall 2024/eCom-Chatbot/notebooks'

In [6]:
os.chdir('../')

In [8]:
load_dotenv()

True

In [9]:
## load the API Keys
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

## Langfuse
os.environ['LANGFUSE_PUBLIC_KEY']=os.getenv("LANGFUSE_PUBLIC_KEY")
os.environ['LANGFUSE_SECRET_KEY']=os.getenv("LANGFUSE_SECRET_KEY")
os.environ['LANGFUSE_HOST']=os.getenv("LANGFUSE_HOST")

## Postgres DB
instance_connection_name = os.getenv("INSTANCE_CONNECTION_NAME")
db_user = os.getenv("DB_USER")  
db_pass = os.getenv("DB_PASS")  
db_name = os.getenv("DB_NAME")  

In [10]:
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler(
    public_key=os.environ['LANGFUSE_PUBLIC_KEY'],
    secret_key=os.environ['LANGFUSE_SECRET_KEY'],
    host=os.environ['LANGFUSE_HOST']
)

In [15]:
mlflow.models.set_model(create_graph(isMemory=True))

In [ ]:
config = {"configurable": {"thread_id": "1"}, "callbacks": [langfuse_handler]}

input_example = {
    "question": "What is the product dimension?", 
    "meta_data": "cache/meta/ABCD-B072K6TLJX.csv",
    "retriever": "cache/faiss/ABCD-B072K6TLJX",
    "config": config
}

with mlflow.start_run() as run_id:
    model_info = mlflow.langchain.log_model(
        lc_model="src/graph.py", # Path to our model Python file
        artifact_path="langgraph",
        pip_requirements="requirements.txt",
    )

    model_uri = model_info.model_uri

2024/11/14 00:54:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [22]:
model_uri

'runs:/24edf641d1204bc1a491929ded90279d/langgraph'

In [23]:
loaded_model = mlflow.langchain.load_model(model_uri)

In [24]:
config = {"configurable": {"thread_id": "1"}, "callbacks": [langfuse_handler]}

In [25]:
payload = {
    "question": "What is the product dimension?", 
    "meta_data": "cache/meta/ABCD-B072K6TLJX.csv",
    "retriever": "cache/faiss/ABCD-B072K6TLJX",
}
response = loaded_model.invoke(payload, config=config)

In [26]:
response['answer'].content

'The product dimensions are 1.01 inches in length, 0.73 inches in width, and 0.57 inches in height.'